In [16]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd
import aesara.tensor.subtensor as st
import aesara.tensor as at

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Prediction of time series

This example demonstrates ...

Adapted from [unit 10: sunspots.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/sunspots.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/sunspots.txt).

Associated lecture video: Unit 10 Lesson 7

## Problem statement

Sunspot numbers observed each year from 1770 to 1869. 

BUGS Book Page 258.

In [3]:
y = np.loadtxt("../data/sunspots.txt")

In [13]:
y

array([100.8,  81.6,  66.5,  34.8,  30.6,   7. ,  19.8,  92.5, 154.4,
       125.9,  84.8,  68.1,  38.5,  22.8,  10.2,  24.1,  82.9, 132. ,
       130.9, 118.1,  89.9,  66.6,  60. ,  46.9,  41. ,  21.3,  16. ,
         6.4,   4.1,   6.8,  14.5,  34. ,  45. ,  43.1,  47.5,  42.2,
        28.1,  10.1,   8.1,   2.5,   0. ,   1.4,   5. ,  12.2,  13.9,
        35.4,  45.8,  41.1,  30.4,  23.9,  15.7,   6.6,   4. ,   1.8,
         8.5,  16.6,  36.3,  49.7,  62.5,  67. ,  71. ,  47.8,  27.5,
         8.5,  13.2,  56.9, 121.5, 138.3, 103.2,  85.8,  63.2,  36.8,
        24.2,  10.7,  15. ,  40.1,  61.5,  98.5, 124.3,  95.9,  66.5,
        64.5,  54.2,  39. ,  20.6,   6.7,   4.3,  22.8,  54.8,  93.8,
        95.7,  77.2,  59.1,  44. ,  47. ,  30.5,  16.3,   7.3,  37.3,
        73.9])

In [14]:
y.shape

(100,)

In [11]:
t = np.array(range(100))
yr = t + 1770

In [15]:
coords = {"t": t, "y": y}

In [4]:
with pm.Model(coords=coords) as m:
    m = 
    
    
    eps_1 = pm.Normal("eps_1", 0, tau=.0001)
    m[1] = y[1] - eps[1]

    theta = pm.Normal("theta", 0, tau=.0001)
    c = pm.Normal("c", 0, tau=.0001)
    sigma = pm.Uniform("sigma", 0, 100)
    tau = 1/(sigma**2)
    
    m = c + theta * y
    eps = y - m

    eps[1] = eps_1
    yr = t + 1769
    
    pm.Normal("likelihood", mu=m, tau=tau)
    
    trace = pm.sample(3000)

TypeError: 'TensorVariable' object does not support item assignment

notes:

- try above way first, but also just found pm.AR under timeseries, so check that out.